In [33]:
import os
import math
import copy
from itertools import zip_longest

import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split

import torch
from torch import nn
from torch import optim

In [34]:
def set_random_seed(state=1):
    gens = (np.random.seed, torch.manual_seed, torch.cuda.manual_seed)
    for set_state in gens:
        set_state(state)

In [35]:
RANDOM_STATE = 42
set_random_seed(RANDOM_STATE)

In [36]:
DATASET_LINK='https://files.grouplens.org/datasets/movielens/ml-32m.zip'

In [37]:
import urllib.request
import zipfile

'''
zip_filename = DATASET_LINK.split('/')[-1]
''This script downloads the MovieLens 32M dataset if it is not already present, and extracts it
from the zip file. The dataset is used for building recommendation systems.''

# Download the dataset if not already present
if not os.path.exists(zip_filename):
	import ssl
	context = ssl._create_unverified_context()
	urllib.request.urlretrieve(DATASET_LINK, zip_filename)

# Unzip the file if not already extracted
with zipfile.ZipFile(zip_filename, 'r') as zip_ref:
	zip_ref.extractall()'''

"\nzip_filename = DATASET_LINK.split('/')[-1]\n''This script downloads the MovieLens 32M dataset if it is not already present, and extracts it\nfrom the zip file. The dataset is used for building recommendation systems.''\n\n# Download the dataset if not already present\nif not os.path.exists(zip_filename):\n\timport ssl\n\tcontext = ssl._create_unverified_context()\n\turllib.request.urlretrieve(DATASET_LINK, zip_filename)\n\n# Unzip the file if not already extracted\nwith zipfile.ZipFile(zip_filename, 'r') as zip_ref:\n\tzip_ref.extractall()"

In [38]:
def read_data(path):
    files = {}
    for filename in os.listdir(path):
        stem, suffix =  os.path.splitext(filename)
        file_path = os.path.join(path,filename)
        print(filename)
        if suffix == '.csv':
            files[stem] = pd.read_csv(file_path)
        elif suffix == '.dat':
            if stem == 'ratings':
                columns = ['userId', 'movieId', 'rating', 'timestamp']
            else:
                columns = ['movieId', 'title', 'genres']
            data = pd.read_csv(file_path, sep='::', names=columns, engine='python')
            files[stem] = data
    return files['ratings'], files['movies']

In [41]:
# Use existing data in data/raw directory (skip download/extraction)
extracted_dir = '/Users/nolanrobbins/Desktop/VS Code Projects/MovieLens-RecSys/data/raw'
print(f"Using existing data from: {extracted_dir}")

# Read data from the raw directory
ratings, movies = read_data(extracted_dir)

Using existing data from: /Users/nolanrobbins/Desktop/VS Code Projects/MovieLens-RecSys/data/raw
links.csv
tags.csv
checksums.txt
ratings.csv
README.txt
movies.csv


In [42]:
ratings.head()

,userId,movieId,rating,timestamp
0,1,17,4.0,944249077
1,1,25,1.0,944250228
2,1,29,2.0,943230976
3,1,30,5.0,944249077
4,1,32,5.0,943228858


In [43]:
minmax = ratings.rating.min(), ratings.rating.max()
print(f"Rating range: {minmax}")

Rating range: (np.float64(0.5), np.float64(5.0))


In [44]:
movies.head()

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy


In [45]:
movies.drop_duplicates().shape

(87585, 3)

In [46]:
ratings = ratings.merge(movies[["movieId", "title"]], on="movieId")

In [47]:
ratings.head()

,userId,movieId,rating,timestamp,title
0,1,17,4.0,944249077,Sense and Sensibility (1995)
1,1,25,1.0,944250228,Leaving Las Vegas (1995)
2,1,29,2.0,943230976,"City of Lost Children, The (Cité des enfants p..."
3,1,30,5.0,944249077,Shanghai Triad (Yao a yao yao dao waipo qiao) ...
4,1,32,5.0,943228858,Twelve Monkeys (a.k.a. 12 Monkeys) (1995)


In [48]:
def tabular_preview(ratings, n=15):
    """Creates a cross-tabular view of users vs movies."""
    
    user_groups = ratings.groupby('userId')['rating'].count()
    top_users = user_groups.sort_values(ascending=False)[:n]

    movie_groups = ratings.groupby('movieId')['rating'].count()
    top_movies = movie_groups.sort_values(ascending=False)[:n]

    top = (
        ratings.
        join(top_users, rsuffix='_r', how='inner', on='userId').
        join(top_movies, rsuffix='_r', how='inner', on='movieId'))

    return pd.crosstab(top.userId, top.movieId, top.rating, aggfunc=np.sum)

In [49]:
tabular_preview(ratings, n=15)

/var/folders/7t/pvkx_dbx3hn67b1jjpbrc37c0000gn/T/ipykernel_20716/3933855387.py:15: FutureWarning: The provided callable <function sum at 0x1211fade0> is currently using DataFrameGroupBy.sum. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "sum" instead.
  return pd.crosstab(top.userId, top.movieId, top.rating, aggfunc=np.sum)


movieId,1,50,110,260,296,318,356,480,527,589,593,1196,2571,2959,4993
userId,,,,,,,,,,,,,,,
7858,5.0,5.0,3.5,4.5,5.0,5.0,2.0,3.0,5.0,3.5,2.5,5.0,5.0,4.0,5.0
10202,5.0,5.0,1.0,5.0,5.0,5.0,4.0,4.0,5.0,4.0,5.0,5.0,5.0,5.0,5.0
14674,3.0,4.5,5.0,5.0,2.0,2.5,5.0,2.0,3.5,5.0,3.0,5.0,4.0,3.5,5.0
17035,3.5,5.0,4.0,0.5,5.0,5.0,3.5,3.0,4.5,3.5,5.0,0.5,1.5,4.5,3.0
22744,5.0,4.0,4.0,5.0,5.0,3.0,4.0,5.0,0.5,5.0,5.0,5.0,5.0,5.0,5.0
49305,5.0,4.5,4.0,5.0,5.0,4.5,4.5,4.0,5.0,5.0,5.0,5.0,5.0,4.0,4.5
53192,NaN,4.0,3.5,5.0,3.5,4.5,4.0,3.0,4.0,3.0,4.0,4.5,2.5,3.0,3.0
55653,4.0,4.5,5.0,4.0,4.5,4.0,4.0,3.0,4.0,4.0,4.0,4.0,4.0,3.0,4.0
57304,4.0,3.0,NaN,2.5,4.0,2.5,NaN,2.0,5.0,3.0,4.0,3.0,4.0,3.5,3.0


In [50]:
def create_dataset(ratings, top=None):
    if top is not None:
        ratings.groupby('userId')['rating'].count()
    
    unique_users = ratings.userId.unique()
    user_to_index = {old: new for new, old in enumerate(unique_users)}
    new_users = ratings.userId.map(user_to_index)
    
    unique_movies = ratings.movieId.unique()
    movie_to_index = {old: new for new, old in enumerate(unique_movies)}
    new_movies = ratings.movieId.map(movie_to_index)
        
    n_users = unique_users.shape[0]
    n_movies = unique_movies.shape[0]
    
    X = pd.DataFrame({'user_id': new_users, 'movie_id': new_movies})
    y = ratings['rating'].astype(np.float32)
    return (n_users, n_movies), (X, y), (user_to_index, movie_to_index)

In [51]:
(n, m), (X, y), (user_to_index, movie_to_index) = create_dataset(ratings)
print(f'Embeddings: {n} users, {m} movies')
print(f'Dataset shape: {X.shape}')
print(f'Target shape: {y.shape}')

Embeddings: 200948 users, 84432 movies
Dataset shape: (32000204, 2)
Target shape: (32000204,)


# Creating the Dataloader

In [52]:
class ReviewsIterator:
    
    def __init__(self, X, y, batch_size=32, shuffle=True):
        X, y = np.asarray(X), np.asarray(y)
        
        if shuffle:
            index = np.random.permutation(X.shape[0])
            X, y = X[index], y[index]
            
        self.X = X
        self.y = y
        self.batch_size = batch_size
        self.shuffle = shuffle
        self.n_batches = int(math.ceil(X.shape[0] // batch_size))
        self._current = 0
        
    def __iter__(self):
        return self
    
    def __next__(self):
        return self.next()
    
    def next(self):
        if self._current >= self.n_batches:
            raise StopIteration()
        k = self._current
        self._current += 1
        bs = self.batch_size
        return self.X[k*bs:(k + 1)*bs], self.y[k*bs:(k + 1)*bs]

In [53]:
def batches(X, y, bs=32, shuffle=True):
    for xb, yb in ReviewsIterator(X, y, bs, shuffle):
        xb = torch.LongTensor(xb)
        yb = torch.FloatTensor(yb)
        yield xb, yb.view(-1, 1) 

In [54]:
for x_batch, y_batch in batches(X, y, bs=4):
    print(x_batch)
    print(y_batch)
    break

tensor([[ 66953,   5323],
        [  9876,   1582],
        [ 38347,    968],
        [101951,    517]])
tensor([[5.],
        [4.],
        [2.],
        [1.]])


In [55]:
# Time-based split: 20% newest data for ETL pipeline testing
timestamp_threshold = ratings['timestamp'].quantile(0.8)
print(f"Timestamp threshold (80th percentile): {timestamp_threshold}")

# Split by timestamp
older_data_mask = ratings['timestamp'] <= timestamp_threshold
newer_data_mask = ratings['timestamp'] > timestamp_threshold

# Get indices for X and y
older_indices = X.index[older_data_mask]
newer_indices = X.index[newer_data_mask]

# Create newer dataset (20% most recent)
X_newer = X.loc[newer_indices].reset_index(drop=True)
y_newer = y.loc[newer_indices].reset_index(drop=True)

# Create older dataset (80% older data)
X_older = X.loc[older_indices].reset_index(drop=True)
y_older = y.loc[older_indices].reset_index(drop=True)

print(f"Newer data (for ETL pipeline): {len(X_newer):,} samples")
print(f"Older data (for train/val split): {len(X_older):,} samples")

# Now do normal train/test split on the older data
X_train, X_valid, y_train, y_valid = train_test_split(
   X_older, y_older, test_size=0.2, random_state=RANDOM_STATE
)

# Organize datasets
datasets = {
   'train': (X_train, y_train),
   'val': (X_valid, y_valid),
   'newer': (X_newer, y_newer)  # For ETL pipeline testing
}

dataset_sizes = {
'train': len(X_train),
   'val': len(X_valid),
   'newer': len(X_newer)
}

print(f"\nFinal split sizes:")
print(f"Training: {dataset_sizes['train']:,}")
print(f"Validation: {dataset_sizes['val']:,}")
print(f"Newer (ETL): {dataset_sizes['newer']:,}")
print(f"Total: {sum(dataset_sizes.values()):,}")

Timestamp threshold (80th percentile): 1538551302.0
Newer data (for ETL pipeline): 6,400,041 samples
Older data (for train/val split): 25,600,163 samples

Final split sizes:
Training: 20,480,130
Validation: 5,120,033
Newer (ETL): 6,400,041
Total: 32,000,204


# Export Training Data

In [56]:
# Export ALL datasets with temporal features for training
print("Exporting datasets with temporal features...")

# We need to get back to the original ratings indices for each split
# The issue is X_train.index doesn't correspond to ratings.index after the splits

# Get the original indices from the splits
train_mask_indices = older_indices[X_train.index]
val_mask_indices = older_indices[X_valid.index] 
test_mask_indices = newer_indices

# Create datasets with full temporal information using original ratings data
train_ratings = ratings.iloc[train_mask_indices].copy()
val_ratings = ratings.iloc[val_mask_indices].copy()
test_ratings = ratings.iloc[test_mask_indices].copy()

print(f"Train ratings shape: {train_ratings.shape}")
print(f"Val ratings shape: {val_ratings.shape}")  
print(f"Test ratings shape: {test_ratings.shape}")

# Add user/movie indices to all datasets
train_ratings['user_idx'] = train_ratings['userId'].map(user_to_index)
train_ratings['movie_idx'] = train_ratings['movieId'].map(movie_to_index)

val_ratings['user_idx'] = val_ratings['userId'].map(user_to_index)
val_ratings['movie_idx'] = val_ratings['movieId'].map(movie_to_index)

test_ratings['user_idx'] = test_ratings['userId'].map(user_to_index)
test_ratings['movie_idx'] = test_ratings['movieId'].map(movie_to_index)

# Add temporal features to all datasets
for name, df in [("train", train_ratings), ("val", val_ratings), ("test", test_ratings)]:
    print(f"Adding temporal features to {name} data...")
    df['rating_date'] = pd.to_datetime(df['timestamp'], unit='s')
    df['rating_year'] = df['rating_date'].dt.year
    df['rating_month'] = df['rating_date'].dt.month
    df['rating_weekday'] = df['rating_date'].dt.weekday
    # Convert to string for CSV compatibility
    df['rating_date'] = df['rating_date'].dt.strftime('%Y-%m-%d %H:%M:%S')

# Define consistent column order
columns = ['userId', 'movieId', 'rating', 'timestamp', 'user_idx', 'movie_idx', 
           'rating_date', 'rating_year', 'rating_month', 'rating_weekday']

# Prepare final datasets
train_df = train_ratings[columns].copy()
val_df = val_ratings[columns].copy()
test_df = test_ratings[columns].copy()

# Create output directory
import os
os.makedirs('data/processed', exist_ok=True)

# Export to CSV files with temporal features
train_df.to_csv('data/processed/train_data.csv', index=False)
val_df.to_csv('data/processed/val_data.csv', index=False)
test_df.to_csv('data/processed/test_data.csv', index=False)

print(f"✅ Training data exported: {len(train_df):,} samples")
print(f"✅ Validation data exported: {len(val_df):,} samples")
print(f"✅ Test data exported: {len(test_df):,} samples")

# Save mappings to processed directory
import pickle
mappings = {
   'user_to_index': user_to_index,
   'movie_to_index': movie_to_index,
   'n_users': n,
   'n_movies': m,
   'minmax': minmax
}

with open('data/processed/data_mappings.pkl', 'wb') as f:
    pickle.dump(mappings, f)

print("✅ Data mappings saved to data/processed/")
print(f"\nFinal column format: {list(train_df.columns)}")
print(f"All datasets now have consistent temporal features!")
print(f"Users: {n}, Movies: {m}")

Exporting datasets with temporal features...
Train ratings shape: (20480130, 5)
Val ratings shape: (5120033, 5)
Test ratings shape: (6400041, 5)
Adding temporal features to train data...
Adding temporal features to val data...
Adding temporal features to test data...
✅ Training data exported: 20,480,130 samples
✅ Validation data exported: 5,120,033 samples
✅ Test data exported: 6,400,041 samples
✅ Data mappings saved to data/processed/

Final column format: ['userId', 'movieId', 'rating', 'timestamp', 'user_idx', 'movie_idx', 'rating_date', 'rating_year', 'rating_month', 'rating_weekday']
All datasets now have consistent temporal features!
Users: 200948, Movies: 84432
